In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dataset = pd.read_csv("/kaggle/input/book-rating-prediction01/Train.csv",encoding="latin")
df = pd.DataFrame(dataset)
df.head()

In [ ]:
result = df.dtypes
for i in range(0, len(result)):
  print(df.columns[i], result[i])

In [ ]:
for i in df.columns:
    print(i,df[i].count())

In [ ]:
new_df = df
new_df.head()

In [ ]:
new_df['year'] = new_df['date_added'].apply(lambda x: x[-4:] if isinstance(x, str) else None)

In [ ]:
new_df['review_text'] = new_df['review_text'].str.lower()

In [ ]:
new_df['review_text']

### HTML Tag and urls removal

In [ ]:
import re

def remove_html_tags_and_urls(text):
    clean = re.sub(r'[\t\n]', '', text)
    clean = re.sub(r'<.*?>', '', clean)
    clean = re.sub(r'https?://\S+|www\.\S+', '', clean)
    clean = re.sub(r'\d+', '', clean)
    return clean

In [ ]:
new_df['review_text'] = new_df['review_text'].apply(remove_html_tags_and_urls)

In [ ]:
new_df['review_text']

### Remove Punctuation

In [ ]:
import string
string.punctuation

In [ ]:
punc = '''!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'''

In [ ]:
def remove_punc(text):
    return text.translate(str.maketrans('','', punc))

In [ ]:
new_df['review_text'] = new_df['review_text'].apply(remove_punc)

In [ ]:
new_df['review_text']

### Stopwords Removal

In [ ]:
from nltk.corpus import stopwords

In [ ]:
stopwords.words('english')

In [ ]:
# def remove_stopwords(text):
#     new_text = []
#     for word in text.split():
#         if word in stopwords.words('english'):
#             new_text.append('')
#         else:
#             new_text.append(word)
#     x = new_text[:]
#     new_text.clear()
#     return " ".join(x)

In [ ]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def remove_stopword(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)


In [ ]:
new_df['review_text'] = new_df['review_text'].apply(remove_stopword)

In [ ]:
new_df['review_text']

### Tokenization and Stemming

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize

In [ ]:
ps = PorterStemmer()
def stemng(text):
    token = word_tokenize(text)
    return " ".join(ps.stem(word) for word in token)

In [ ]:
new_df['review_text'] = new_df['review_text'].apply(stemng)

In [ ]:
new_df['review_text']

### Model

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd
import scipy.sparse as sp

# Split the data into features and target
X = new_df[['review_text', 'n_votes', 'n_comments']]
y = new_df['rating']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# TF-IDF vectorization for text data
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train['review_text'])
X_test_tfidf = vectorizer.transform(X_test['review_text'])

# Combine TF-IDF vectors with numerical columns
X_train_combined = sp.hstack((X_train_tfidf, X_train[['n_votes', 'n_comments']].values))
X_test_combined = sp.hstack((X_test_tfidf, X_test[['n_votes', 'n_comments']].values))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Fit logistic regression model (multinomial)
logistic_model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
logistic_model.fit(X_train_tfidf, y_train)

# Predict on the test set
predictions = logistic_model.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(report)


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# TF-IDF vectorization for text data
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train['review_text'])
X_test_tfidf = vectorizer.transform(X_test['review_text'])


# Fit Naive Bayes (Multinomial) model
nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf, y_train)

# Predict on the test set
predictions = nb_model.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(report)



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import scipy.sparse as sp


# Split the data into features and target
X = new_df[['review_text', 'n_votes', 'n_comments']]
y = new_df['rating']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# TF-IDF vectorization for text data
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train['review_text'])
X_test_tfidf = vectorizer.transform(X_test['review_text'])

# Combine TF-IDF vectors with numerical columns
X_train_combined = sp.hstack((X_train_tfidf, X_train[['n_votes', 'n_comments']].values))
X_test_combined = sp.hstack((X_test_tfidf, X_test[['n_votes', 'n_comments']].values))

# Fit SVM model
svm_model = SVC(kernel='linear')  # You can choose different kernels like 'rbf', 'poly', etc.
svm_model.fit(X_train_combined, y_train)

# Predict on the test set
predictions = svm_model.predict(X_test_combined)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(report)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import scipy.sparse as sp



# Split the data into features and target
X = new_df[['review_text', 'n_votes', 'n_comments']]
y = new_df['rating']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# TF-IDF vectorization for text data
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train['review_text'])
X_test_tfidf = vectorizer.transform(X_test['review_text'])

# Combine TF-IDF vectors with numerical columns
X_train_combined = sp.hstack((X_train_tfidf, X_train[['n_votes', 'n_comments']].values))
X_test_combined = sp.hstack((X_test_tfidf, X_test[['n_votes', 'n_comments']].values))

# Fit Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_combined, y_train)

# Predict on the test set
predictions = rf_model.predict(X_test_combined)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(report)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import scipy.sparse as sp


# Split the data into features and target
X = new_df[['review_text', 'n_votes', 'n_comments']]
y = new_df['rating']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# TF-IDF vectorization for text data
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train['review_text'])
X_test_tfidf = vectorizer.transform(X_test['review_text'])

# Combine TF-IDF vectors with numerical columns
X_train_combined = sp.hstack((X_train_tfidf, X_train[['n_votes', 'n_comments']].values))
X_test_combined = sp.hstack((X_test_tfidf, X_test[['n_votes', 'n_comments']].values))

# Fit KNN model
knn_model = KNeighborsClassifier(n_neighbors=750)  # Adjust the number of neighbors as needed
knn_model.fit(X_train_combined, y_train)

# Predict on the test set
predictions = knn_model.predict(X_test_combined)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(report)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import scipy.sparse as sp


# Split the data into features and target
X = new_df[['review_text', 'n_votes', 'n_comments']]
y = new_df['rating']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# TF-IDF vectorization for text data
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train['review_text'])
X_test_tfidf = vectorizer.transform(X_test['review_text'])

# Combine TF-IDF vectors with numerical columns
X_train_combined = sp.hstack((X_train_tfidf, X_train[['n_votes', 'n_comments']].values))
X_test_combined = sp.hstack((X_test_tfidf, X_test[['n_votes', 'n_comments']].values))

# Fit MLPClassifier (Neural Network)
mlp_model = MLPClassifier(hidden_layer_sizes=(100, ), max_iter=1000, random_state=42)
mlp_model.fit(X_train_combined, y_train)

# Predict on the test set
predictions = mlp_model.predict(X_test_combined)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(report)


### BERT

In [ ]:
from bert import tokenization
import tensorflow as tf
import tensorflow_hub as hub
from keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [ ]:
m_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
bert_layer = hub.KerasLayer(m_url, trainable=True)

In [ ]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
        
        text = text[:max_len-2]
        input_sequence = text 

        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        
        
        all_tokens.append(tokens)
        
    return np.array(all_tokens)

In [ ]:
def build_model(bert_layer, max_len=512):
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
    
    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    
    clf_output = sequence_output[:, 0, :]
    
    lay = tf.keras.layers.Dense(64, activation='relu')(clf_output)
    lay = tf.keras.layers.Dropout(0.2)(lay)
    lay = tf.keras.layers.Dense(32, activation='relu')(lay)
    lay = tf.keras.layers.Dropout(0.2)(lay)
    out = tf.keras.layers.Dense(5, activation='softmax')(lay)
    
    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=2e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
import absl.flags as flags

# Reset or clear FLAGS before using the tokenizer
flags.FLAGS = flags.FlagValues()


In [ ]:
max_len = 250
train_input = bert_encode(new_df['review_text'].tolist(), tokenizer, max_len=max_len)

train_labels = new_df['rating']

In [ ]:
model = build_model(bert_layer, max_len=max_len)
model.summary()

In [ ]:
train_input = new_df['review_text']
train_labels = new_df['rating']

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

train_sh = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=3,
    callbacks=[checkpoint, earlystopping],
    batch_size=32,
    verbose=1
)


In [ ]:
def bert_encode_single_input(texts, tokenizer, max_len=512):
    all_tokens = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
        text = text[:max_len - 2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        all_tokens.append(tokens)
        
    return np.array(all_tokens)

# Encode using the modified function
max_len = 250
train_input = bert_encode_single_input(new_df['review_text'], tokenizer, max_len=max_len)
train_labels = new_df['rating']

### Prediction

In [ ]:
test_df = pd.read_csv('/kaggle/input/book-rating-prediction01/Test.csv')

### Preprocessing Testing Data

In [ ]:
test_df['review_text'] = test_df['review_text'].str.lower()

In [ ]:
test_df['review_text'] = test_df['review_text'].apply(remove_html_tags_and_urls)

In [ ]:
test_df['review_text'] = test_df['review_text'].apply(remove_punc)

In [ ]:
test_df['review_text'] = test_df['review_text'].apply(remove_stopword)

In [ ]:
test_df['review_text'] = test_df['review_text'].apply(stemng)

In [ ]:
Y_test = vectorizer.transform(test_df['review_text'])

In [ ]:
predictions = logistic_model.predict(Y_test)

In [ ]:
predictions_df = pd.DataFrame({'review_id': test_df['review_id'], 'rating': predictions})
predictions_df.to_csv('predictions.csv', index=False)

In [ ]:
test_df['review_text']